Check the HEF implementation of NLL and normalisation constant 

In [2]:
import numpy as np
from scipy.stats import multivariate_normal

In [3]:
import os 
import sys 
base_path = os.path.dirname(os.path.dirname(os.getcwd()))
sys.path.append(base_path + "/local/HarmonicExponentialBayesFitler")

In [4]:
from src.filters.bayes_filter import BayesFilter 
from src.distributions.se2_distributions import SE2Gaussian
from src.sampler.se2_sampler import se2_grid_samples
from lie_learn.spectral.SE2FFT import SE2_FFT

In [5]:
size = (50, 50, 32)
fft = SE2_FFT(spatial_grid_size=size,
              interpolation_method='spline',
              spline_order=1,
              oversampling_factor=1)

In [6]:
poses, x, y, yaw = se2_grid_samples(size)
mu = np.array([0.1, 0.2, np.pi / 2])
cov = np.diag([0.05, 0.05, 0.05])
# Define distribution
gaussian = SE2Gaussian(mu=mu, cov=cov, samples=poses, fft=fft)

> /network/scratch/r/ria.arora/local/HarmonicExponentialBayesFitler/src/distributions/se2_distributions.py(26)from_samples()
     24         pdb.set_trace()
     25         # Compute energy of samples
---> 26         energy = self.compute_energy(self.samples)
     27         _, _, _, _, _, self.eta = self.fft.analyze(energy)
     28         # This seems redundant, but as there is a loss of information in FFT analyze due to cartesian to polar



ipdb>  c


In [64]:
from typing import Tuple
def se2_grid_samples(size: Tuple[int] = (5, 5, 5),
                     lower_bound: float = -0.5,
                     upper_bound: float = 0.5) -> np.ndarray:
    xs = np.linspace(lower_bound, upper_bound, size[0], endpoint=False)
    ys = np.linspace(lower_bound, upper_bound, size[1], endpoint=False)
    ts = np.linspace(0., 2. * np.pi, size[2], endpoint=False)
    X, Y, T = np.meshgrid(xs, ys, ts, indexing='ij')
    poses = np.vstack((X.flatten(), Y.flatten(), T.flatten())).T
    return poses, X, Y, T

In [65]:
poses_np, _, _, _ = se2_grid_samples(size)

In [7]:
gaussian.normalize()

In [19]:
energy = gaussian.energy
prob = gaussian.prob


In [21]:
inv_cov = np.linalg.inv(cov)
diff = poses - mu

# Wrap the angular difference to [-pi, pi]
diff[:, 2] = (diff[:, 2] + np.pi) % (2 * np.pi) - np.pi

# Compute the energy
quadratic_form = np.sum((diff @ inv_cov) * diff, axis=1)
energy = -0.5 * quadratic_form
energy = energy.reshape(size)

In [22]:
_, _, _, _, _, fh = fft.analyze(energy)

#### Check the normalisation constant

In [10]:
gaussian.l_n_z

-0.1735137111999463

In [36]:
d = cov.shape[0]  # Dimensionality of the Gaussian distribution
det_cov = np.linalg.det(cov)  # Determinant of the covariance matrix

# Normalization constant formula
constant = np.log((2 * np.pi) ** (d / 2) * np.sqrt(det_cov))
print(constant)

-1.7367828107169683


In [23]:
_, l_n_z = gaussian.compute_moments_lnz(fh,True)

In [83]:
l_n_z

-1.9839514169577657

For cov = 0.01, the diff between HEF normalisation constant and the gaussian normalisation constant is 0.18200 and for cov = 0.05 the diff is 0.253 and it gets worse for cov=0.1

In [11]:
poses_check = np.vstack((mu, mu+0.025, mu+0.05, mu+0.075, mu-0.1))

In [38]:
prob_true = multivariate_normal.logpdf(poses_check, mean=mu, cov=cov)
print(prob_true)

[1.73678281 1.71803281 1.66178281 1.56803281 1.43678281]


In [37]:
prob_true + constant

array([ 2.22044605e-16, -1.87500000e-02, -7.50000000e-02, -1.68750000e-01,
       -3.00000000e-01])

In [ ]:
p_energy = np.exp(-BayesFilter.neg_log_likelihood(gaussian.eta, gaussian.l_n_z, poses, fft))
# p_m = -BayesFilter.neg_log_likelihood(gaussian.M, 0.0, poses, fft)
# p_moments = -BayesFilter.neg_log_likelihood(gaussian.moments, 0.0, poses, fft)

In [9]:
gaussian.eta.shape

(37, 223, 32)

In [34]:
-BayesFilter.neg_log_likelihood(gaussian.eta, 0, mu+0.05, fft) + gaussian.l_n_z

array([-13.15503082])

In [41]:
-BayesFilter.neg_log_likelihood2(energy,0, mu-0.1, size )

array([-0.39634954])

Check the bilinear interpolation

In [12]:
import numpy as np
def bilinear_interpolate(f, x, y):
    x = np.asarray(x)
    y = np.asarray(y)

    x0 = np.floor(x).astype(int)
    x1 = x0 + 1
    y0 = np.floor(y).astype(int)
    y1 = y0 + 1

    x0 = np.clip(x0, 0, f.shape[1] - 1)
    x1 = np.clip(x1, 0, f.shape[1] - 1)
    y0 = np.clip(y0, 0, f.shape[0] - 1)
    y1 = np.clip(y1, 0, f.shape[0] - 1)

    Ia = f[y0, x0]
    Ib = f[y1, x0]
    Ic = f[y0, x1]
    Id = f[y1, x1]

    wa = (x1 - x) * (y1 - y)
    wb = (x1 - x) * (y - y0)
    wc = (x - x0) * (y1 - y)
    wd = (x - x0) * (y - y0)

    print(x0.shape, y0.shape, x1.shape, y1.shape, x.shape, y.shape)
    print(Ia.shape, Ib.shape, Ic.shape, Id.shape)
    print(wa.shape, wb.shape, wc.shape, wd.shape)

#     return wa[..., None] * Ia + wb[..., None] * Ib + wc[..., None] * Ic + wd[..., None] * Id

    return wa * Ia + wb * Ib + wc * Ic + wd * Id


In [15]:
f = np.array([
    [1, 2, 3],
    [4, 5, 6],
    [7, 8, 9]
], dtype=np.float32)

# Points to interpolate
x = np.array([ 2.0])
y = np.array([ 0.5])

# Perform bilinear interpolation
result = bilinear_interpolate(f, x, y)
print("Interpolated Values:", result)

(1,) (1,) (1,) (1,) (1,) (1,)
(1,) (1,) (1,) (1,)
(1,) (1,) (1,) (1,)
Interpolated Values: [0.]


In [10]:
import torch

def bilinear_interpolate_torch(f, x, y):
    """
    Perform bilinear interpolation on a 2D tensor using PyTorch.
    
    Parameters:
        f (torch.Tensor): Input tensor of shape (H, W).
        x (torch.Tensor): x-coordinates to interpolate (float tensor).
        y (torch.Tensor): y-coordinates to interpolate (float tensor).
    
    Returns:
        torch.Tensor: Interpolated values at the given coordinates.
    """
    # Ensure x and y are tensors
    x = x.to(f.device)
    y = y.to(f.device)

    # Get integer grid points
    x0 = torch.floor(x).long()
    x1 = x0 + 1
    y0 = torch.floor(y).long()
    y1 = y0 + 1

    # Clip to valid range
    x0 = torch.clamp(x0, 0, f.shape[1] - 1)
    x1 = torch.clamp(x1, 0, f.shape[1] - 1)
    y0 = torch.clamp(y0, 0, f.shape[0] - 1)
    y1 = torch.clamp(y1, 0, f.shape[0] - 1)

    # Gather values from the 2D tensor
    Ia = f[y0, x0]  # Bottom-left
    Ib = f[y1, x0]  # Top-left
    Ic = f[y0, x1]  # Bottom-right
    Id = f[y1, x1]  # Top-right

    # Compute the weights for interpolation
    wa = (x1 - x) * (y1 - y)  # Weight for Ia
    wb = (x1 - x) * (y - y0)  # Weight for Ib
    wc = (x - x0) * (y1 - y)  # Weight for Ic
    wd = (x - x0) * (y - y0)  # Weight for Id
    
    print(x0.shape, y0.shape, x1.shape, y1.shape, x.shape, y.shape)
    print(Ia.shape, Ib.shape, Ic.shape, Id.shape)
    print(wa.shape, wb.shape, wc.shape, wd.shape)

    # Compute the interpolated value
    return wa * Ia + wb * Ib + wc * Ic + wd * Id


In [14]:
# Input 2D tensor (e.g., image or matrix)
f = torch.tensor([
    [1, 2, 3],
    [4, 5, 6],
    [7, 8, 9]
], dtype=torch.float32)

# Coordinates to interpolate
x = torch.tensor([2.0])
y = torch.tensor([0.5])

# Perform interpolation
result = bilinear_interpolate_torch(f, x, y)
print("Interpolated Values:", result)


torch.Size([1]) torch.Size([1]) torch.Size([1]) torch.Size([1]) torch.Size([1]) torch.Size([1])
torch.Size([1]) torch.Size([1]) torch.Size([1]) torch.Size([1])
torch.Size([1]) torch.Size([1]) torch.Size([1]) torch.Size([1])
Interpolated Values: tensor([0.])


In [40]:
import torch

def bilinear_interpolate_torch_with_nan(f, x, y):
    """
    Perform bilinear interpolation on a 2D tensor with `NaN` padding in PyTorch.
    
    Parameters:
        f (torch.Tensor): Input tensor of shape (H, W).
        x (torch.Tensor): x-coordinates to interpolate (float tensor).
        y (torch.Tensor): y-coordinates to interpolate (float tensor).
    
    Returns:
        torch.Tensor: Interpolated values at the given coordinates.
    """
    # Pad the tensor with NaN
    f = torch.nn.functional.pad(f, (1, 1, 1, 1), mode='constant', value=float('nan'))

    # Adjust coordinates for the padded tensor
    x = x + 1
    y = y + 1

    # Get integer grid points
    x0 = torch.floor(x).long()
    x1 = x0 + 1
    y0 = torch.floor(y).long()
    y1 = y0 + 1

    # Gather values from the 2D tensor
    Ia = f[y0, x0]  # Bottom-left
    Ib = f[y1, x0]  # Top-left
    Ic = f[y0, x1]  # Bottom-right
    Id = f[y1, x1]  # Top-right
    
    print(f"Ia {Ia}, Ib {Ib}, Ic {Ic}, Id {Id}")
    # Compute the weights for interpolation
    wa = (x1 - x) * (y1 - y)  # Weight for Ia
    wb = (x1 - x) * (y - y0)  # Weight for Ib
    wc = (x - x0) * (y1 - y)  # Weight for Ic
    wd = (x - x0) * (y - y0)  # Weight for Id
    
    print(f"wa {wa}, wb {wb}, wc {wc}, wd {wd}")

    # Mask out NaN values
    valid_a = ~torch.isnan(Ia)
    valid_b = ~torch.isnan(Ib)
    valid_c = ~torch.isnan(Ic)
    valid_d = ~torch.isnan(Id)
    
    print(f"valid_a {valid_a}, valid_b {valid_b}, valid_c {valid_c}, valid_d {valid_d}")
    

    # Set weights to zero where values are NaN
    wa = wa * valid_a
    wb = wb * valid_b
    wc = wc * valid_c
    wd = wd * valid_d

    # Normalize weights to handle missing data
    weight_sum = wa + wb + wc + wd
    print("weight_sum", weight_sum)
    
    Ia = torch.where(valid_a, Ia, torch.zeros_like(Ia))
    Ib = torch.where(valid_b, Ib, torch.zeros_like(Ib))
    Ic = torch.where(valid_c, Ic, torch.zeros_like(Ic))
    Id = torch.where(valid_d, Id, torch.zeros_like(Id))
    
#     wa = wa / (weight_sum + 1e-8)
#     wb = wb / (weight_sum + 1e-8)
#     wc = wc / (weight_sum + 1e-8)
#     wd = wd / (weight_sum + 1e-8)

    # Compute the interpolated value
    result = wa * Ia + wb * Ib + wc  * Ic + wd  * Id
    print("result",result)
    result[weight_sum == 0] = float('nan')  # If all weights are zero, output NaN

    return result


In [41]:
f = torch.tensor([
    [1, 2, 3],
    [4, 5, 6],
    [7, 8, 9]
], dtype=torch.float32)

# Coordinates to interpolate
x = torch.tensor([0.5])
y = torch.tensor([2])

# Perform interpolation with NaN padding
result = bilinear_interpolate_torch_with_nan(f, x, y)
print("Interpolated Values:", result)

Ia tensor([7.]), Ib tensor([nan]), Ic tensor([8.]), Id tensor([nan])
wa tensor([0.5000]), wb tensor([0.]), wc tensor([0.5000]), wd tensor([0.])
valid_a tensor([True]), valid_b tensor([False]), valid_c tensor([True]), valid_d tensor([False])
weight_sum tensor([1.])
result tensor([7.5000])
Interpolated Values: tensor([7.5000])


In [44]:
import numpy as np

def bilinear_interpolate_with_nan_safe(f, x, y):
    """
    Perform bilinear interpolation on a 2D NumPy array with `NaN` padding,
    safely handling `NaN` values during the computation.

    Parameters:
        f (np.ndarray): Input 2D array of shape (H, W).
        x (np.ndarray): x-coordinates to interpolate (float array).
        y (np.ndarray): y-coordinates to interpolate (float array).

    Returns:
        np.ndarray: Interpolated values at the given coordinates.
    """
    # Pad the array with NaN
    f = np.pad(f, ((1, 1), (1, 1)), mode='constant', constant_values=np.nan)

    # Adjust coordinates for the padded array
    x = np.asarray(x) + 1
    y = np.asarray(y) + 1

    # Get integer grid points
    x0 = np.floor(x).astype(int)
    x1 = x0 + 1
    y0 = np.floor(y).astype(int)
    y1 = y0 + 1

    # Clip to ensure indices are within bounds
    x0 = np.clip(x0, 0, f.shape[1] - 1)
    x1 = np.clip(x1, 0, f.shape[1] - 1)
    y0 = np.clip(y0, 0, f.shape[0] - 1)
    y1 = np.clip(y1, 0, f.shape[0] - 1)

    # Gather values from the 2D array
    Ia = f[y0, x0]  # Bottom-left
    Ib = f[y1, x0]  # Top-left
    Ic = f[y0, x1]  # Bottom-right
    Id = f[y1, x1]  # Top-right

    # Compute the weights for interpolation
    wa = (x1 - x) * (y1 - y)  # Weight for Ia
    wb = (x1 - x) * (y - y0)  # Weight for Ib
    wc = (x - x0) * (y1 - y)  # Weight for Ic
    wd = (x - x0) * (y - y0)  # Weight for Id

    # Mask NaN values
    valid_a = ~np.isnan(Ia)
    valid_b = ~np.isnan(Ib)
    valid_c = ~np.isnan(Ic)
    valid_d = ~np.isnan(Id)

    # Set invalid contributions to zero
    Ia = np.where(valid_a, Ia, 0)
    Ib = np.where(valid_b, Ib, 0)
    Ic = np.where(valid_c, Ic, 0)
    Id = np.where(valid_d, Id, 0)

    # Adjust weights for NaN values
    wa = wa * valid_a
    wb = wb * valid_b
    wc = wc * valid_c
    wd = wd * valid_d

    # Normalize weights to handle missing data
    weight_sum = wa + wb + wc + wd
#     normalized_wa = wa / (weight_sum + 1e-8)
#     normalized_wb = wb / (weight_sum + 1e-8)
#     normalized_wc = wc / (weight_sum + 1e-8)
#     normalized_wd = wd / (weight_sum + 1e-8)

    # Compute the interpolated value
    result = (
        wa * Ia +
        wb * Ib +
        wc * Ic +
        wd * Id
    )

    # If all weights are zero, set result to NaN
    result[weight_sum == 0] = np.nan

    return result


In [45]:
# Input array
f = np.array([
    [1, 2, 3],
    [4, 5, 6],
    [7, 8, 9]
], dtype=np.float64)

# Coordinates to interpolate
x = np.array([1.5, 2.0])
y = np.array([1.5, 0.5])

# Perform bilinear interpolation
result = bilinear_interpolate_with_nan_safe(f, x, y)
print("Interpolated Values:", result)


Interpolated Values: [7.  4.5]
